# Prédiction d'anomalies sur une machine

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## 1. Import des librairies

In [ ]:
# Général
import pandas as pd
import numpy as np
import os

#Préprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Modèles
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Evaluateurs
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

## 2. Import des données

In [ ]:
df = pd.DataFrame(columns=[1, 2, 3, 4, 5, 6, 7])

for i, (root, dirs, files) in enumerate(os.walk('Data')):

    if i ==0:
        output = dirs

    for j, file in enumerate(files):

        if j == 0:
          temp = pd.read_csv(root + '/' + file, names = [1, 2, 3, 4, 5, 6, 7], index_col=False )
          temp['y'] = output[i-1]
          
          df = pd.concat([df, temp]).reset_index(drop = True)

## 3. Visualisation des données

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

## 4. Split

In [ ]:
np.random.RandomState(42)

X = df.drop('y', axis = 1)

y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

y_train.head()

## 5. Préprocessing

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)

## 6. Logistic Regression

In [ ]:
logReg = LogisticRegression(solver = 'lbfgs')

logReg.fit(X_test, y_test)

y_pred = logReg.predict(X_test)


print(f"Matrice de confusion: \n{confusion_matrix(y_test, y_pred)}\n")

print(f"Precision: {round(accuracy_score(y_test, y_pred), 3) * 100}%\n")

print(classification_report(y_test, y_pred))

## 7. XGBOOST

In [ ]:
XGBOOST_model = XGBClassifier(n_estimators = 100, objective = "multi:softmax")

In [ ]:
XGBOOST_model.fit(X_train, y_train, verbose=True)

XGBOOST_pred = XGBOOST_model.predict(X_test)

In [ ]:
print(f"Matrice de confusion: \n{confusion_matrix(y_test, XGBOOST_pred)}\n")
print(f"Precision: {round(accuracy_score(y_test, XGBOOST_pred), 4) * 100}%\n")
print(classification_report(y_test, XGBOOST_pred))

## 8. Neural Network

In [ ]:
NN_model = Sequential()

NN_model.add(Dense(20, input_dim = X_train.shape[1]))
NN_model.add(Dense(10))
NN_model.add(Dense(1))

NN_model.compile(loss='mean_absolute_error', optimizer = Adam(learning_rate=0.001))

In [ ]:
NN_model.fit(X_train, y_train, epochs=10, verbose=1)

In [ ]:
NN_pred = NN_model.predict(X_test)

print(f"Matrice de confusion: \n{confusion_matrix(y_test, NN_pred)}\n")
print(f"Precision: {round(accuracy_score(y_test, NN_pred), 4) * 100}%\n")
print(classification_report(y_test, NN_pred))

In [ ]:
NN_pred